In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import urg_func
import ejecuta_modelo
import urg_func
from datetime import date

In [6]:
connection = sqlite3.connect("data/BDTerra.db")
crsr = connection.cursor()

def sql_query(query):    # Ejecuta la query
    crsr.execute(query)    # Almacena los datos de la query
    ans = crsr.fetchall()    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]    
    return pd.DataFrame(ans,columns=names)


In [8]:
query1 = """
SELECT *
FROM ISSUES
"""

query2 = """
SELECT * 
FROM Projects
"""

df_orig = sql_query(query1)

df_project = sql_query(query2)


df_orig["Input Date"] = pd.to_datetime(df_orig["Input Date"])

In [9]:
dic = {"Issue ID": ["IPC10010111"],"Project ID": ["PC1001"],"Input Date": ["2025-06-10"],"Device": ["Desktop"],"Page": ["Home"]}
df = pd.DataFrame(dic)
df

,Issue ID,Project ID,Input Date,Device,Page
0,IPC10010111,PC1001,2025-06-10,Desktop,Home


In [36]:
df_project.columns

Index(['Project ID', 'Name', 'Project Type', 'Client ID', 'Budget',
       'Project Manager ID', 'Beggining date', 'Contact ID'],
      dtype='object')

In [42]:
def inicio_proyecto(proj_id):
    inicio = pd.to_datetime(df_project[df_project["Project ID"] == proj_id]["Beggining date"].values[0])
    print(inicio)
    return inicio

def timelapse(issue_id):
    fecha = df[df["Issue ID"] == issue_id]["Input Date"].values[0]
    fecha = pd.to_datetime(fecha)
    print(fecha,type(fecha))
    proj_id = df[df["Issue ID"] == issue_id]["Project ID"].values[0]
    date1 = fecha - pd.DateOffset(days=30)
    print(date1)
    ini = inicio_proyecto(proj_id)
    print(ini)
    res = max(date1, ini)
    return res

def suma_it(issue_id):
    fecha_30 = timelapse(issue_id)
    # print(fecha_30)
    input = df[df["Issue ID"] == issue_id]["Input Date"].values[0]
    # print(input)
    proj_id = df[df["Issue ID"] == issue_id]["Project ID"].values[0]
    aux = df_orig[(df_orig["Project ID"] == proj_id) & (df_orig["Input Date"].between(fecha_30,input))]
    # print(aux)
    return int(aux["Iteraciones"].sum())

In [ ]:
issue_type = {"Copy issues": "E1",
              "Design issues": "E2",
              "Request change": "E2",
              "New item": "E3",
              "Not addressing": "E4",
              "Bug fix": "E3"}

def nueva_entrada(lectura):
    dic = {
            "Issue ID": [lectura["Issue ID"]],
            "Project ID": [lectura["Project ID"]],
            "Classifiation": [lectura["Classification"]],
            "Screenshot": [bool(lectura["Screenshot"])],
            "Urgency": [None],
            "Input Date": [lectura["Input Date"]],
            "Deadline Theor": [None],
            "Deadline Real": [None],
            "Employee ID": [issue_type[lectura["Classification"]]],
            "Device": [lectura["Device"]],
            "Browser": [lectura["Browser"]],
            "Page": [lectura["Page"]],
            "Contact ID": [lectura["Contact ID"]],
            "Request": [lectura["Request"]],
            "Iteraciones": [0],
            "Iteraciones 30 dias": [None],
            "Not addressing historico": [None],
            "Sentiment": [None],
            "Sentiment historico": [None]
           }
    return dic


### GENERAMOS DATAFRAME CON LA ENTRADA

dic = {"Issue ID": "IPC10010111","Project ID": "PC1001","Input Date": "2025-06-10","Device": "Desktop","Page": "Home",
       "Browser": "Chrome", "Contact ID": "COC101", "Request": "This is a very good job","Classification": "Bug fix",
       "Screenshot": True}
# print("Antes dataframe")
entrada = nueva_entrada(dic)
# print(entrada.keys())
df = pd.DataFrame(entrada)
# print("Kapasao")

dict_keys(['Issue ID', 'Project ID', 'Classifiation', 'Screenshot', 'Urgency', 'Input Date', 'Deadline Theor', 'Deadline Real', 'Employee ID', 'Device', 'Browser', 'Page', 'Contact ID', 'Request', 'Iteraciones', 'Iteraciones 30 dias', 'Not addressing historico', 'Sentiment', 'Sentiment historico'])


In [41]:
df_orig

,Issue ID,Project ID,Classification,Screenshot,Urgency,Input Date,Deadline Theor,Deadline Real,Employee ID,Device,Browser,Page,Contact ID,Request
0,IPC805024,PC805,Bug fix,0,,2022-06-01,,2022-06-02,E3,Desktop,Edge,True,COC801,The language switcher shows incorrect content ...
1,IPC101013,PC101,Bug fix,0,,2018-06-17,,2018-06-19,E3,Tablet,Safari,True,COC101,The login form error handling does not respect...
2,IPC703070,PC703,Bug fix,0,,2015-04-18,,2015-04-19,E3,Desktop,Safari,True,COC701,The image carousel arrows does not render prop...
3,IPC907020,PC907,Bug fix,0,,2018-07-21,,2018-07-24,E3,Desktop,Safari,True,COC901,The lazy loading for images does not respect s...
4,IPC1002078,PC1002,Bug fix,0,,2014-05-08,,2014-05-10,E3,Desktop,Edge,True,COC1001,The FAQ accordion doesn't reset fields correctly.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,IPC10408,PC104,Request change,0,,2021-02-27,,2021-02-28,E2,Desktop,Mozilla,True,COC101,"We'd like to change the menu order on tablet, ..."
5696,IPC901106,PC9011,Request change,0,,2022-12-01,,2022-12-02,E2,Desktop,Chrome,True,COC901,We'd like to change the positioning of the tes...
5697,IPC909037,PC909,Request change,0,,2013-10-12,,2013-10-13,E2,Desktop,Chrome,True,COC901,"We'd like to change the CTA icon placement, si..."
5698,IPC9011011,PC9011,Request change,0,,2023-06-10,,2023-06-11,E2,Desktop,Chrome,True,COC901,We'd like to change the download link style in...


In [43]:
sumas = []
for i in df["Issue ID"]:
    print("sumas", i)
    sumas.append(suma_it(i))
sumas

sumas IPC10010111
2025-06-10 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2025-05-11 00:00:00
2014-09-14 00:00:00
2014-09-14 00:00:00


KeyError: 'Iteraciones'